In [1]:
import xml.etree.ElementTree as ET
import pandas as pd
import requests
from bs4 import BeautifulSoup
import io
import csv

In [2]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

# Step 1: Fetch the product page
product_url = "https://www.ishares.com/us/products/333011/ishares-bitcoin-trust"
response = requests.get(product_url, headers=headers)

In [3]:
soup = BeautifulSoup(response.text, 'html.parser')
holdings_link = None

# Look for the specific link by class or text
for a in soup.find_all('a', href=True):
    if 'detailed holdings and analytics' in a.text.lower():
        holdings_link = a['href']
        break

In [4]:
holdings_link = "https://www.ishares.com" + holdings_link

In [5]:
file_response = requests.get(holdings_link, headers=headers)

In [8]:
preamble = file_response.text.splitlines()[:8]
shares_outstanding = None
for line in csv.reader(preamble):
	if line and "Shares Outstanding" in line[0]:
		shares_outstanding = line[1].strip('"')
		break

In [ ]:
df = pd.read_csv(io.StringIO(file_response.text), skiprows=9)

# Extract BTC Held from Quantity column
btc_row = df[df['Ticker'] == 'BTC']
if not btc_row.empty:
	btc_held = btc_row['Quantity'].values[0]
	print(f"BTC Held: {btc_held}")
